In [1]:
import pandas as pd

# import data

In [2]:
train_identity = pd.read_csv('~/data/train_identity.csv')
train_transaction = pd.read_csv('~/data/train_transaction.csv')
test_identity = pd.read_csv('~/data/test_identity.csv')
test_transaction = pd.read_csv('~/data/test_transaction.csv')
sample = pd.read_csv('~/data/sample_submission.csv')

In [3]:
print('train_transaction shape is {}'.format(train_transaction.shape))
print('test_transaction shape is {}'.format(test_transaction.shape))
print('train_identity shape is {}'.format(train_identity.shape))
print('test_identity shape is {}'.format(test_identity.shape))

train_transaction shape is (590540, 394)
test_transaction shape is (506691, 393)
train_identity shape is (144233, 41)
test_identity shape is (141907, 41)


In [4]:
list(set(train_transaction.columns) - set(test_transaction.columns))

['isFraud']

In [5]:
train_transaction = train_transaction.drop('isFraud', axis=1)

In [6]:
test_identity.columns = ['TransactionID','id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08',
       'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16',
       'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24',
       'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo']

In [7]:
transaction_data = pd.concat([train_transaction, test_transaction])
identity_data = pd.concat([train_identity, test_identity])

In [8]:
print('transaction_data shape is {}'.format(transaction_data.shape))
print('identity_data shape is {}'.format(identity_data.shape))

transaction_data shape is (1097231, 393)
identity_data shape is (286140, 41)


# clean data

## Create a list of numerical and categorical features 

### in identity_data

In [43]:
# create a list of numerical features
# create a list of categorical features

c = (identity_data.dtypes == 'object')
n = (identity_data.dtypes != 'object')
cat_id_cols = list(c[c].index)
num_id_cols = list(n[n].index) 

print(cat_id_cols, "\n")
print("number categorical identity features: ", len(cat_id_cols), "\n\n")
print(num_id_cols, "\n")
print("number numerical identity features: ", len(num_id_cols))

['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo'] 

number categorical identity features:  17 


['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_19', 'id_20', 'id_32'] 

number numerical identity features:  16


### in transaction_data

In [44]:
# create a list of numerical features
# create a list of categorical features

c = (transaction_data.dtypes == 'object')
n = (transaction_data.dtypes != 'object')
cat_trans_cols = list(c[c].index)
num_trans_cols = list(n[n].index) 

print(cat_trans_cols, "\n")
print("number categorical transaction features: ", len(cat_trans_cols), "\n\n")
print(num_trans_cols, "\n")

['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'] 

number categorical transaction features:  14 


['TransactionID', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76

## create low, medium, many missing values

### for identity

#### for cat

In [45]:
low_missing_cat_id_cols = []      # lower than 15% missing values
medium_missing_cat_id_cols = []   # between 15% and 60% missing
many_missing_cat_id_cols = []     # more than 60% missing

for i in cat_id_cols:
    percentage = identity_data[i].isnull().sum() * 100 / len(identity_data[i])
    if percentage < 15:
        low_missing_cat_id_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_cat_id_cols.append(i)
    else:
        many_missing_cat_id_cols.append(i)

#### for num

In [46]:
low_missing_num_id_cols = []      # lower than 15% missing values
medium_missing_num_id_cols = []   # between 15% and 60% missing
many_missing_num_id_cols = []     # more than 60% missing

for i in num_id_cols:
    percentage = identity_data[i].isnull().sum() * 100 / len(identity_data[i])
    if percentage < 15:
        low_missing_num_id_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_num_id_cols.append(i)
    else:
        many_missing_num_id_cols.append(i)

### for transaction

#### for cat

In [47]:
low_missing_cat_trans_cols = []      # lower than 15% missing values
medium_missing_cat_trans_cols = []   # between 15% and 60% missing
many_missing_cat_trans_cols = []     # more than 60% missing

for i in cat_trans_cols:
    percentage = transaction_data[i].isnull().sum() * 100 / len(transaction_data[i])
    if percentage < 15:
        low_missing_cat_trans_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_cat_trans_cols.append(i)
    else:
        many_missing_cat_trans_cols.append(i)

#### for num

In [48]:
low_missing_num_trans_cols = []      # lower than 15% missing values
medium_missing_num_trans_cols = []   # between 15% and 60% missing
many_missing_num_trans_cols = []     # more than 60% missing

for i in num_trans_cols:
    percentage = transaction_data[i].isnull().sum() * 100 / len(transaction_data[i])
    if percentage < 15:
        low_missing_num_trans_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_num_trans_cols.append(i)
    else:
        many_missing_num_trans_cols.append(i)

## Clean NUMERICAL features

### drop features with many missing values

#### for identity

In [49]:
print("shape before dropping num_id_cols: ", identity_data.shape, "\n")        
identity_data = identity_data.drop(columns = many_missing_num_id_cols)
print("shape after dropping num_id_cols: ", identity_data.shape, "\n")

shape before dropping num_id_cols:  (286140, 33) 

shape after dropping num_id_cols:  (286140, 33) 



#### for transaction

In [50]:
print("shape before dropping num_trans_cols: ", transaction_data.shape, "\n")        
transaction_data = transaction_data.drop(columns = many_missing_num_trans_cols)
print("shape after dropping num_trans_cols: ", transaction_data.shape, "\n\n")  

shape before dropping num_trans_cols:  (1097231, 393) 

shape after dropping num_trans_cols:  (1097231, 226) 




### Impute features with low missing values

#### for identity

In [53]:
print("index before imputation: ", identity_data.index, "\n")
print("columns before imputation: ", identity_data.columns, "\n")


my_imputer = SimpleImputer(strategy = 'mean') 
my_imputer.fit(identity_data[low_missing_num_id_cols])
identity_data[low_missing_num_id_cols] = my_imputer.transform(identity_data[low_missing_num_id_cols])

print("index after imputation: ", identity_data.index, "\n")
print("columns after imputation: ", identity_data.columns, "\n")

index before imputation:  Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            141897, 141898, 141899, 141900, 141901, 141902, 141903, 141904,
            141905, 141906],
           dtype='int64', length=286140) 

columns before imputation:  Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16',
       'id_17', 'id_19', 'id_20', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object') 

index after imputation:  Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            141897, 141898, 141899, 141900, 141901, 141902, 141903, 141904,
            141905, 141906],
           dtype='int64', length=286140) 

columns af

#### for transaction

In [52]:
from sklearn.impute import SimpleImputer

print("index before imputation: ", transaction_data.index, "\n")
print("columns before imputation: ", transaction_data.columns, "\n")

my_imputer = SimpleImputer(strategy = 'mean') 
my_imputer.fit(transaction_data[low_missing_num_trans_cols])
transaction_data[low_missing_num_trans_cols] = my_imputer.transform(transaction_data[low_missing_num_trans_cols])

print("index after imputation: ", transaction_data.index, "\n")
print("columns after imputation: ", transaction_data.columns, "\n")

index before imputation:  Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            506681, 506682, 506683, 506684, 506685, 506686, 506687, 506688,
            506689, 506690],
           dtype='int64', length=1097231) 

columns before imputation:  Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320',
       'V321'],
      dtype='object', length=226) 

index after imputation:  Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            506681, 506682, 506683, 506684, 506685, 506686, 506687, 506688,
            506689, 506690],
           dtype='int64', length=1097231) 

columns after imputation:  Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'c

### impute features with medium missing values

#### for identity

In [55]:
print("index before imputation: ", identity_data.index, "\n")
print("columns before imputation: ", identity_data.columns, "\n")


my_imputer = SimpleImputer(strategy = 'median') 
my_imputer.fit(identity_data[medium_missing_num_id_cols])

print("values before imputing: ", identity_data[medium_missing_num_id_cols], "\n")

identity_data[medium_missing_num_id_cols] = my_imputer.transform(identity_data[medium_missing_num_id_cols])

print("index after imputation: ", identity_data.index, "\n")
print("columns after imputation: ", identity_data.columns, "\n")

index before imputation:  Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            141897, 141898, 141899, 141900, 141901, 141902, 141903, 141904,
            141905, 141906],
           dtype='int64', length=286140) 

columns before imputation:  Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16',
       'id_17', 'id_19', 'id_20', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object') 

values before imputing:          id_03  id_04  id_09  id_10  id_14  id_32
0         NaN    NaN    NaN    NaN -480.0   32.0
1         NaN    NaN    NaN    NaN -300.0   32.0
2         0.0    0.0    0.0    0.0    NaN    NaN
3         NaN    NaN    NaN    NaN    NaN    NaN
4         0.0    0.0    0.0    0.

#### for transaction

In [54]:
print("index before imputation: ", transaction_data.index, "\n")
print("columns before imputation: ", transaction_data.columns, "\n")

print("values before imputing: ", transaction_data[medium_missing_num_trans_cols], "\n")

my_imputer = SimpleImputer(strategy = 'median') 
my_imputer.fit(transaction_data[medium_missing_num_trans_cols])

transaction_data[medium_missing_num_trans_cols] = my_imputer.transform(transaction_data[medium_missing_num_trans_cols])

print("index after imputation: ", transaction_data.index, "\n")
print("columns after imputation: ", transaction_data.columns, "\n")

index before imputation:  Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            506681, 506682, 506683, 506684, 506685, 506686, 506687, 506688,
            506689, 506690],
           dtype='int64', length=1097231) 

columns before imputation:  Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320',
       'V321'],
      dtype='object', length=226) 

values before imputing:          dist1     D2     D3     D4     D5    D11   V1   V2   V3   V4  ...  \
0        19.0    NaN   13.0    NaN    NaN   13.0  1.0  1.0  1.0  1.0  ...   
1         NaN    NaN    NaN    0.0    NaN    NaN  NaN  NaN  NaN  NaN  ...   
2       287.0    NaN    NaN    0.0    NaN  315.0  1.0  1.0  1.0  1.0  ...   
3         NaN  112.0    0.0   94.0    0.0    NaN  NaN  NaN  NaN  NaN 

index after imputation:  Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            506681, 506682, 506683, 506684, 506685, 506686, 506687, 506688,
            506689, 506690],
           dtype='int64', length=1097231) 

columns after imputation:  Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320',
       'V321'],
      dtype='object', length=226) 



## Clean CATEGORICAL features

### drop features with many missing values

In [58]:
print("shape before dropping many_missing_cat_trans_cols: ", transaction_data.shape, "\n")        
transaction_data = transaction_data.drop(columns = many_missing_cat_trans_cols)
print("shape after dropping many_missing_cat_trans_cols: ", transaction_data.shape, "\n\n")    

print("shape before dropping many_missing_cat_id_cols: ", identity_data.shape, "\n")        
identity_data = identity_data.drop(columns = many_missing_cat_id_cols)
print("shape after dropping many_missing_cat_id_cols: ", identity_data.shape, "\n")


# because we dropped some categorical columns from the dataframe,
# we must create the list 'cat_trans_cols' and
# 'cat_id_cols' again such that the dropped cols are no longer in them
c = (transaction_data.dtypes == 'object')
cat_trans_cols = list(c[c].index) 

c = (identity_data.dtypes == 'object')
cat_id_cols = list(c[c].index) 



shape before dropping many_missing_cat_trans_cols:  (1097231, 226) 

shape after dropping many_missing_cat_trans_cols:  (1097231, 224) 


shape before dropping many_missing_cat_id_cols:  (286140, 33) 

shape after dropping many_missing_cat_id_cols:  (286140, 31) 



# merge data

In [60]:
df_merged = transaction_data.merge(identity_data, on='TransactionID', how='left')

In [61]:
df_merged.shape

(1097231, 254)